# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 16 2022 3:14PM,244305,10,MSP213161,"Methapharm, Inc.",Released
1,Aug 16 2022 3:14PM,244305,10,MSP213169,"Methapharm, Inc.",Released
2,Aug 16 2022 2:58PM,244304,10,0085939217,ISDIN Corporation,Released
3,Aug 16 2022 2:58PM,244304,10,0085939218,ISDIN Corporation,Released
4,Aug 16 2022 2:58PM,244304,10,0085939221,ISDIN Corporation,Released
5,Aug 16 2022 2:58PM,244304,10,0085939219,ISDIN Corporation,Released
6,Aug 16 2022 2:58PM,244304,10,0085939220,ISDIN Corporation,Released
7,Aug 16 2022 2:58PM,244304,10,0085939242,ISDIN Corporation,Released
8,Aug 16 2022 2:58PM,244304,10,0085939243,ISDIN Corporation,Released
9,Aug 16 2022 2:58PM,244304,10,0085939244,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,244300,Released,8
38,244302,Released,1
39,244303,Released,6
40,244304,Released,29
41,244305,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244300,NaN,NaN,8.0
244302,NaN,NaN,1.0
244303,NaN,NaN,6.0
244304,NaN,NaN,29.0
244305,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244192,0.0,6.0,6.0
244194,0.0,0.0,1.0
244198,0.0,0.0,1.0
244215,47.0,10.0,17.0
244216,19.0,2.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244192,0,6,6
244194,0,0,1
244198,0,0,1
244215,47,10,17
244216,19,2,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244192,0,6,6
1,244194,0,0,1
2,244198,0,0,1
3,244215,47,10,17
4,244216,19,2,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244192,,6,6
1,244194,,,1
2,244198,,,1
3,244215,47,10,17
4,244216,19,2,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc."
2,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation
31,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation
37,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions
38,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation
46,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd"
65,Aug 16 2022 1:59PM,244298,12,Hush Hush
67,Aug 16 2022 1:38PM,244290,10,Beach Products Inc
68,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance
72,Aug 16 2022 1:22PM,244288,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",,,2
1,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation,,,29
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,,,6
3,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,,,1
4,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,,,8
5,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",,4,15
6,Aug 16 2022 1:59PM,244298,12,Hush Hush,,,2
7,Aug 16 2022 1:38PM,244290,10,Beach Products Inc,,,1
8,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance,,,4
9,Aug 16 2022 1:22PM,244288,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",2,,
1,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation,29,,
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6,,
3,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1,,
4,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8,,
5,Aug 16 2022 2:04PM,244299,19,"AdvaGen Pharma, Ltd",15,4,
6,Aug 16 2022 1:59PM,244298,12,Hush Hush,2,,
7,Aug 16 2022 1:38PM,244290,10,Beach Products Inc,1,,
8,Aug 16 2022 1:32PM,244289,10,Beach Patient Assistance,4,,
9,Aug 16 2022 1:22PM,244288,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",2,,
1,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation,29,,
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6,,
3,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1,,
4,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",2.0,NaN,NaN
1,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation,29.0,NaN,NaN
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6.0,NaN,NaN
3,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 16 2022 3:14PM,244305,10,"Methapharm, Inc.",2.0,0.0,0.0
1,Aug 16 2022 2:58PM,244304,10,ISDIN Corporation,29.0,0.0,0.0
2,Aug 16 2022 2:57PM,244303,10,ISDIN Corporation,6.0,0.0,0.0
3,Aug 16 2022 2:49PM,244302,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Aug 16 2022 2:33PM,244300,10,ISDIN Corporation,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3175589,78.0,18.0,2.0
12,488581,2.0,1.0,0.0
15,732653,40.0,12.0,72.0
16,977112,4.0,0.0,0.0
19,244299,15.0,4.0,0.0
20,977028,50.0,9.0,0.0
21,488392,2.0,0.0,0.0
22,732772,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3175589,78.0,18.0,2.0
1,12,488581,2.0,1.0,0.0
2,15,732653,40.0,12.0,72.0
3,16,977112,4.0,0.0,0.0
4,19,244299,15.0,4.0,0.0
5,20,977028,50.0,9.0,0.0
6,21,488392,2.0,0.0,0.0
7,22,732772,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,78.0,18.0,2.0
1,12,2.0,1.0,0.0
2,15,40.0,12.0,72.0
3,16,4.0,0.0,0.0
4,19,15.0,4.0,0.0
5,20,50.0,9.0,0.0
6,21,2.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,12,Released,2.0
2,15,Released,40.0
3,16,Released,4.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,1.0,12.0,0.0,4.0,9.0,0.0,0.0
Released,78.0,2.0,40.0,4.0,15.0,50.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,1.0,12.0,0.0,4.0,9.0,0.0,0.0
2,Released,78.0,2.0,40.0,4.0,15.0,50.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,1.0,12.0,0.0,4.0,9.0,0.0,0.0
2,Released,78.0,2.0,40.0,4.0,15.0,50.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()